# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [27]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import json
from IPython.display import Markdown, display, update_display

In [7]:
import gradio as gr # oh yeah!

In [8]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBA


In [9]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [5]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [6]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [7]:
message_gpt("What is today's date?")

"Today's date is October 5, 2023."

## User Interface time!

In [8]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [9]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [15]:
# The simplicty of gradio. This might appear in "light mode" - I'll show you how to make this in dark mode later.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Shout has been called with input hello


In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer. There is a way to force gradio to appear in dark mode, but Gradio recommends against this as it should be a user preference (particularly for accessibility reasons). But if you wish to force dark mode for your screens, below is how to do it.

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
# Inputs and Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# And now - changing the function from "shout" to "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Let's use Markdown
# Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
# Not a great software engineering practice, but quite sommon during Jupyter Lab R&D!

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

## Minor improvement

I've made a small improvement to this code.

Previously, it had these lines:

```
for chunk in result:
  yield chunk
```

There's actually a more elegant way to achieve this (which Python people might call more 'Pythonic'):

`yield from result`

I cover this in more detail in the Intermediate Python notebook in the week1 folder - take a look if you'd like more.

In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

# Building a company brochure generator

Now you know how - it's simple!

In [11]:
## This prompt changes the company to a person.

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a personal website \
and creates a short brochure about the person for prospective friends, coworkers, and employers. Respond in markdown.\
Include details of careers/jobs and personal interests if you have the information."

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the person, \
such as links to an About page, or a Resume page, or Publications pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [25]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the person, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

def get_links(url,model="GPT"):
    website = Website(url)
    if model == "GPT":
        response = openai.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": get_links_user_prompt(website)}
          ],
            response_format={"type": "json_object"}
        )
        result = response.choices[0].message.content
        return json.loads(result)
    elif model == "Claude":
        message = claude.messages.create(
         model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=link_system_message,
        messages=[
            {"role": "user", "content": get_links_user_prompt(website)},
        ],)
        result = message.content[0].text
        return json.loads(result)
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

def get_brochure_user_prompt(person_name, url):
    user_prompt = f"You are looking at a person called: {person_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short introduction to the person in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:1_000] # Truncate if more than 1,000 characters
    return user_prompt

def create_brochure(person_name, url, model='GPT'):
    if model == 'GPT':
        response = openai.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": get_brochure_user_prompt(person_name, url)}
              ],
        )
        result = response.choices[0].message.content
    elif model == 'Claude':
        message = claude.messages.create(
         model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=[
            {"role": "user", "content": get_brochure_user_prompt(person_name, url)},
        ],)
        result = message.content[0].text
    display(Markdown(result))

def stream_brochure(person_name, url, model):
    prompt = f"Please generate a personal brochure for {person_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [30]:
create_brochure("Lara Pudwell","https://faculty.valpo.edu/lpudwell/","Claude")

Found links: {'links': [{'type': 'about page', 'url': 'https://faculty.valpo.edu/lpudwell/aboutme.html'}, {'type': 'work page', 'url': 'https://faculty.valpo.edu/lpudwell/work.html'}, {'type': 'education page', 'url': 'https://faculty.valpo.edu/lpudwell/education.html'}, {'type': 'papers page', 'url': 'https://faculty.valpo.edu/lpudwell/papers.html'}, {'type': 'presentations page', 'url': 'https://faculty.valpo.edu/lpudwell/presentations.html'}, {'type': 'CV', 'url': 'https://faculty.valpo.edu/lpudwell/cv_october2024.pdf'}]}


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Lara K. Pudwell

Lara K. Pudwell is a Full Professor of Mathematics and Statistics and the Dixon W. and Herta E. Benz Professor at Valparaiso University. She has an extensive academic and professional background in the field of mathematics.

## Career and Education

- Full Professor of Mathematics and Statistics at Valparaiso University
- Dixon W. and Herta E. Benz Professor at Valparaiso University
- Holds a Ph.D. in Mathematics from Emory University

## Professional Memberships and Activities

- Active member of various professional organizations in mathematics and statistics
- Regularly gives talks and presentations at conferences and events

## Personal Interests

While the website does not provide detailed information about Lara's personal interests, it does suggest that she is deeply passionate about her work in mathematics and statistics. The website features a wide range of information about her academic and professional achievements, including her publications, research projects, and involvement in various mathematical organizations.

Overall, Lara K. Pudwell appears to be a highly accomplished and dedicated scholar in the field of mathematics and statistics, with a strong commitment to teaching, research, and professional service.

In [32]:
view = gr.Interface(
    fn=create_brochure,
    inputs=[
        gr.Textbox(label="Person name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")]
)
view.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://58a93c177fdef9773b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Found links: {'links': [{'type': 'about page', 'url': 'https://faculty.valpo.edu/lpudwell/aboutme.html'}, {'type': 'work page', 'url': 'https://faculty.valpo.edu/lpudwell/work.html'}, {'type': 'education page', 'url': 'https://faculty.valpo.edu/lpudwell/education.html'}, {'type': 'papers page', 'url': 'https://faculty.valpo.edu/lpudwell/papers.html'}, {'type': 'presentations page', 'url': 'https://faculty.valpo.edu/lpudwell/presentations.html'}, {'type': 'curriculum vitae', 'url': 'https://faculty.valpo.edu/lpudwell/cv_october2024.pdf'}]}


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Introducing Lara K. Pudwell

## About Lara
Lara K. Pudwell is a distinguished Full Professor of Mathematics and Statistics, holding the esteemed Dixon W. and Herta E. Benz Professorship at Valparaiso University. With a solid academic background and a passion for teaching, Lara is committed to fostering a love for mathematics and statistics in her students.

## Contact Information
- **Address:**  
  Department of Mathematics & Statistics  
  Gellersen Center  
  1900 Chapel Drive  
  Valparaiso, IN 46383

- **Office:** Gellersen Center 222  
- **Telephone:** 219-464-5414  
- **Email:** [lara.pudwell@valpo.edu](mailto:lara.pudwell@valpo.edu)

## Educational Background
Lara has an impressive educational pedigree, which she details in her site, including her mathematical genealogy that connects her to influential figures in the field.

## Professional Experience
As a Full Professor, Lara has extensive teaching experience in mathematics and statistics. Her curriculum vitae provides insight into her career trajectory, showcasing her commitment to higher education and her contributions to her field.

## Research and Publications
Lara is not just a teacher but also an active researcher. Her website features various publications and papers, highlighting her contributions to mathematical research. Additionally, she is known for her participation in various academic talks and conferences, further disseminating her work and insights.

## Special Interests
Beyond her academic pursuits, Lara has a rich collection of resources on her website, including *Maple packages*, projects, and a list of her professional memberships. She also shares mathematical quotations and insights, reflecting her deep love for mathematics.

## In the Community
Stay tuned for news and updates on Lara's work, teaching discussions, and engagement with the broader mathematical community. Whether you're a prospective friend, coworker, or employer, Lara's dedication to mathematics and education will surely inspire collaboration and connection.

---

Feel free to reach out via the contact details provided for more information or to connect!

Found links: {'links': [{'type': 'about page', 'url': 'https://faculty.valpo.edu/lpudwell/aboutme.html'}, {'type': 'curriculum vitae', 'url': 'https://faculty.valpo.edu/lpudwell/cv_october2024.pdf'}, {'type': 'work page', 'url': 'https://faculty.valpo.edu/lpudwell/work.html'}, {'type': 'education page', 'url': 'https://faculty.valpo.edu/lpudwell/education.html'}, {'type': 'papers page', 'url': 'https://faculty.valpo.edu/lpudwell/papers.html'}, {'type': 'presentations page', 'url': 'https://faculty.valpo.edu/lpudwell/presentations.html'}]}


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Lara K. Pudwell

Lara K. Pudwell is a Full Professor of Mathematics and Statistics and the Dixon W. and Herta E. Benz Professor at Valparaiso University. She has a wide range of professional and personal interests, as evident from her comprehensive website.

## Professional Background

Lara has an extensive academic career. She is a Full Professor at Valparaiso University, where she has been teaching mathematics and statistics since joining the faculty. Her research interests span various areas of mathematics, and she has an impressive publication record, including numerous journal articles and conference proceedings.

## Education

Lara holds a Ph.D. in Mathematics from the University of Illinois at Urbana-Champaign, where she also earned her Master's degree. She completed her undergraduate studies at the University of Wisconsin-Eau Claire, graduating with a Bachelor's degree in Mathematics.

## Personal Interests

Beyond her academic pursuits, Lara has a diverse range of personal interests. She is actively involved in various professional memberships and organizations, and she enjoys giving talks and presentations on mathematical topics. Lara also maintains a section on her website dedicated to mathematical quotations, showcasing her passion for the field.

## Conclusion

Lara K. Pudwell is a highly accomplished mathematician and educator, with a strong commitment to her work and a wide range of personal interests. Her website provides a comprehensive overview of her professional and academic achievements, as well as insights into her personal life and passions.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you read the next few cells</h2>
            <span style="color:#900;">
                Try to do this yourself - go back to the company brochure in week1, day5 and add a Gradio UI to the end. Then come and look at the solution.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [ ]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=create_brochure,
    inputs=[
        gr.Textbox(label="Person name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()